<h1 style="line-height:.1;"><center>Tugas Akhir Kecerdasan Buatan TIF-A SA</center></h1>
<h1 style="line-height:.5;"><center>Robot Pengangkut Barang Otomatis Berbasis Q-Learning</center></h1>

---

<p1 style="line-height:5;"><center>Oleh Kelompok 2 :</center></p1>
<p1 style="line-height:5;"><center>Dimas Tri Mustakim (205150200111049)</center></p1>
<p1 style="line-height:5;"><center>Salsabila Dita Prasetya (205150201111024)</center></p1>

# Latar Belakang

Sekarang ini kebutuhan untuk transportasi barang merupakan hal yang tidak kalah pentingnya di dalam dunia industri. Transportasi disini tidak hanya dalam jarak jauh saja, tetapi kebutuhan untuk jarak dekat. Di dalam suatu perusahaan seperti perusahaan manufaktur, arus perpindahan barang terjadi sangat cepat. Barang atau bahan produksi perlu dipindahkan setiap saat ke proses berikutnya untuk membuat suatu produk. Sekarang proses tersebut masih banyak yang melibatkan manusia di dalamnya, tetapi hal itu tidak bisa dilakukan untuk barang-barang yang sangat berat, barang berbahaya (racun, radiasi, dll) dan barang yang mudah rusak jika terkontaminasi. Karena hal itulah kita perlu membuat cara baru untuk transportasi barang, salah satunya adalah dengan menggunakan robot dengan Artificial Intelligence di dalamnya.

### Contoh aplikasi agen pengangkut barang
##### 1. Robot warehouse

<img src="https://cdn.vox-cdn.com/thumbor/FnXcyoYv1x5muxd1fY-MdkvnJAI=/0x0:3000x2000/2070x1164/filters:focal(1204x757:1684x1237):format(webp)/cdn.vox-cdn.com/uploads/chorus_image/image/59668735/ocado_robot_smaller.0.png" alt="robot warehouse" width="700"/>

##### 2. Robot transportasi intralogistik

<img src="https://software.intel.com/content/dam/develop/external/us/en/images/seit-amr-from-milvus-robotics-749347.png" alt="intralogistic robot" width="700"/>

# Program

## Membuat Lingkungan

Lingkungan ini dibuat berdasarkan lingkungan dari OpenAI Gym yakni "Taxi-v3".

### Deskripsi lingkungan:

Terdapat lingkungan berbentuk persegi (grid) sebesar 10x10. Di lingkungan tersebut terdapat 4 buah lokasi yang ditandakan dengan huruf R, G, Y, dan B. Barang akan muncul secara random di salah satu tempat tersebut dan tujuan barang juga di salah satu tempat tersebut. Ketika episode dimulai, robot akan mulai di posisi acak di dalam grid. Robot bertugas untuk menjemput barang, mengambil, dan kemudian mengantarkannya ke tempat tujuan. Dalam berjalan, robot tidak dapat melewati garis '|'. Episode berakhir ketika barang telah selesai diantarkan.
    
### Observasi :

Terdapat 2000 keadaan diskrit yang didapat dari 100 kemungkinan posisi taxi (10 x 10), 5 lokasi yang mungkin untuk barang, dan 4 tujuan pengiriman barang.
    
Lokasi barang :
- 0: R
- 1: G
- 2: Y
- 3: B
- 4: Di bawa robot
    
Tujuan antar:
- 0: R
- 1: G
- 2: Y
- 3: B
    
### Actions:

Terdapat 6 aksi yang bersifat deterministic yang dapat diambil oleh agen
    
- 0: jalan selatan
- 1: jalan ke utara
- 2: jalan ke timur
- 3: jalan ke barat
- 4: ambil barang
- 5: taruh barang

### Rewards

1. Agen akan dihadiahi sejumlah -1 setiap step/langkah yang dilakukan. Kecuali :
2. Ketika barang telah dikirimkan ke tempatnya akan dihadiahi poin 30.
3. Ketika melakukan aksi pickup atau drop-off secara illegal, akan dikenai poin -10
    
    
### Rendering:

- blue: barang yang hendak diambil
- magenta: tujuan barang
- yellow: robot yang belum membawa barang
- green: robot yang sedang membawa barang
- cyan: barrier atau elemen yang tidak bisa dilewati
- other letters (R, G, Y and B): lokasi dari barang dan tujuannya
    
### Keadaan lingkungan direpresentasikan dengan format

(robot_row, robot_col, lokasi_barang, tujuan) yang nantinya akan di encode ke dalam satu nilai diskrit

### Import library yang dibutuhkan

In [2]:
import os, sys, time, random
import numpy as np
from IPython.display import clear_output
from contextlib import closing
from io import StringIO

from gym import utils
from gym.envs.toy_text import discrete
from time import sleep

### Peta lingkungan tempat robot berinteraksi

In [3]:
# Peta lingkungan/environment
# Karakter '|' dan ':' dapat direplace satu sama lain untuk memodifikasi

MAP = [
    "+-------------------+",
    "|R| | : | : | : : :G|",
    "| | | | : : | : : : |",
    "| | : | : : | : | : |",
    "| | : | : : | : | : |",
    "| | : | : : | : | : |",
    "| | : | : |B| : | : |",
    "| : : | : : : : | : |",
    "| | : | : : : : | : |",
    "| | : | : : : : | : |",
    "|Y| : : : : | : : : |",
    "+-------------------+",
]

### Membuat lingkungan

Kode merupakan hasil modifikasi dari lingkungan [Taxi milik OpenAI Gym](https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py).

License dari OpenAI Gym : [license](https://github.com/openai/gym/blob/master/LICENSE.md)

Kelas untuk robot ini kami namai **DeliveryRobotEnv**

In [12]:
class DeliveryRobotEnv(discrete.DiscreteEnv):
    
    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self):
        self.desc = np.asarray(MAP, dtype='c')

        self.locs = locs = [(0, 0), (0, 9), (9, 0), (5, 5)]

        num_states = 2000  # 10 * 10 * 5 * 4
        num_rows = 10
        num_columns = 10
        max_row = num_rows - 1
        max_col = num_columns - 1
        initial_state_distrib = np.zeros(num_states)
        num_actions = 6
        P = {state: {action: [] for action in range(num_actions)} for state in range(num_states)}

        for row in range(num_rows):
            for col in range(num_columns):
                for goods_idx in range(len(locs) + 1):
                    for dest_idx in range(len(locs)):
                        state = self.encode(row, col, goods_idx, dest_idx)
                        if goods_idx < 4 and goods_idx != dest_idx:
                            initial_state_distrib[state] += 1
                        for action in range(num_actions):
                            new_row, new_col, new_goods_idx = row, col, goods_idx
                            reward = -1 
                            done = False
                            robot_loc = (row, col)

                            if action == 0:
                                new_row = min(row + 1, max_row)
                            elif action == 1:
                                new_row = max(row - 1, 0)
                            if action == 2 and self.desc[1 + row, 2 * col + 2] == b":":
                                new_col = min(col + 1, max_col)
                            elif action == 3 and self.desc[1 + row, 2 * col] == b":":
                                new_col = max(col - 1, 0)
                            elif action == 4:  # pickup
                                if goods_idx < 4 and robot_loc == locs[goods_idx]:
                                    new_goods_idx = 4
                                else:  # goods not at location
                                    reward = -10
                            elif action == 5:  # dropoff
                                if (robot_loc == locs[dest_idx]) and goods_idx == 4:
                                    new_goods_idx = dest_idx
                                    done = True
                                    reward = 30
                                elif (robot_loc in locs) and goods_idx == 4:
                                    new_goods_idx = locs.index(robot_loc)
                                else:  # dropoff at wrong location
                                    reward = -10
                            new_state = self.encode(new_row, new_col, new_goods_idx, dest_idx)
                            P[state][action].append((1.0, new_state, reward, done))

        initial_state_distrib /= initial_state_distrib.sum()
        discrete.DiscreteEnv.__init__(self, num_states, num_actions, P, initial_state_distrib)

    def encode(self, robot_row, robot_col, goods_loc, dest_idx):
        """
        Berfungsi untuk menencode keadaan lingkungan yang berupa (robot_row, robot_col, lokasi barang, tujuan)
        ke dalam suatu angka diskrit untuk merepresentasikan keadaan lingkungan
        
        4 data keadaan lingkungan tersebut akan di encode ke dalam nilai diskrit di antara 0 sampai 2000
        
        """
        i = robot_row
        i *= 10
        i += robot_col
        i *= 5
        i += goods_loc
        i *= 4
        i += dest_idx
        return i

    def decode(self, i):
        """
        Untuk men-decode angka representasi state kembali ke bentuk (robot_row, robot_col, lokasi_barang, tujuan)
        """
        out = []
        out.append(i % 4)
        i = i // 4
        out.append(i % 5)
        i = i // 5
        out.append(i % 10)
        i = i // 10
        out.append(i)
        assert 0 <= i < 10
        return reversed(out)

    def step(self, a):
        assert 0 <= a < 2000
        return super().step(a)

    def render(self, mode='human'):
        outfile = StringIO() if mode == 'ansi' else sys.stdout

        out = self.desc.copy().tolist()
        out = [[c.decode('utf-8') for c in line] for line in out]
        robot_row, robot_col, goods_idx, dest_idx = self.decode(self.s)

        # Coloring barrier ( '|', '+', '-' )
        for a in out:
            for b in range(len(a)):
                if a[b] == '|' or a[b] == '+' or a[b] == '-':
                    a[b] = utils.colorize(a[b], 'cyan')

        def ul(x):
            return "_" if x == " " else x

        if goods_idx < 4:
            out[1 + robot_row][2 * robot_col + 1] = utils.colorize(
                out[1 + robot_row][2 * robot_col + 1], 'yellow', highlight=True)
            pi, pj = self.locs[goods_idx]
            out[1 + pi][2 * pj + 1] = utils.colorize(out[1 + pi][2 * pj + 1], 'blue', bold=True)
        else:  # passenger in taxi
            out[1 + robot_row][2 * robot_col + 1] = utils.colorize(
                ul(out[1 + robot_row][2 * robot_col + 1]), 'green', highlight=True)

        di, dj = self.locs[dest_idx]
        out[1 + di][2 * dj + 1] = utils.colorize(out[1 + di][2 * dj + 1], 'magenta')
        outfile.write("\n".join(["".join(row) for row in out]) + "\n")
        if self.lastaction is not None:
            outfile.write("  ({})\n".format(
                ["Selatan", "Utara", "Timur", "Barat", "Ambil Barang", "Taruh Barang"][self.lastaction])
            )
            
        else:
            outfile.write("\n")

        if mode != 'human':
            with closing(outfile):
                return outfile.getvalue()


In [13]:
env = DeliveryRobotEnv()
print("Observation space : ", env.observation_space)
print("Action space : ", env.action_space)

Observation space :  Discrete(2000)
Action space :  Discrete(6)


>Disini lingkungan memiliki tipe **Observation Space** dan **Action Space** diskrit. 
>
>Dengan kata lain, lingkungan akan mengembalikan nilai diskrit ke agen sebagai nilai yang merepresentasikan keadaan lingkungan. Di program ini terdapat 2000 (0-1999) nilai observation space. Agen membuat nilai observation space in dengan menggunakan method encode.
>
>Agen kemudian dapat merespon dengan aksi berupa nilai diskrit 0 sampai 5.

In [14]:
# Mencoba untuk men encode suatu keadaan berikut: 

# robot_row : 5
# robot_col : 5
# lokasi barang : 2 
# tujuan : 3

env.encode(5,5,2,3)

1111

In [15]:
# sekarang kita coba decode kembali
robot_row, robot_col, lokasi_barang, tujuan = env.decode(1111)
print(f"posisi_robot_row : {robot_row}\nposisi_robot_col : {robot_col}\nlokasi barang : {lokasi_barang}\ntujuan : {tujuan}")

posisi_robot_row : 5
posisi_robot_col : 5
lokasi barang : 2
tujuan : 3


In [16]:
# kita coba render tampilan keadaan lingkungan seperti keadaan yang dimaksud
env.s = env.encode(5,5,2,3)
env.render()

+-------------------+
|R| | : | : | : : :G|
| | | | : : | : : : |
| | : | : : | : | : |
| | : | : : | : | : |
| | : | : : | : | : |
| | : | : |B| : | : |
| : : | : : : : | : |
| | : | : : : : | : |
| | : | : : : : | : |
|Y| : : : : | : : : |
+-------------------+



### Mengetes lingkungan yang dibuat dengan random action

Disini kita mencoba lingkungan yang telah dibuat menggunakan agen yang memutuskan langkahnya secara random

In [17]:
# Mengetes lingkungan yang telah dibuat

jumlah_episode = 3
max_step = 50

for episode in range(jumlah_episode):
    obs = env.reset()
    score = 0
    done = False
    step = 0

    while not done and step != max_step:
        
        action = env.action_space.sample()
        clear_output(wait=True)
        
        print(f"Episode ke : {episode+1}")
        print(f"Observatian space / State : {obs}")
        print(f"Number of Steps : {step}")
        
        env.render()
        obs, reward, done, info = env.step(action)
        sleep(0.15)
        score += reward
        step += 1
              
    print(f"\nResult\nEpisode : {episode+1}\nScore : {score}")
    status = "Sukses" if done else "Belum Sukses"
    print(f"Status : {status}")
    sleep(2)

env.close()


Episode ke : 3
Observatian space / State : 627
Number of Steps : 49
+-------------------+
|R| | : | : | : : :G|
| | | | : : | : : : |
| | : | : : | : | : |
| | : | : : | : | : |
| | : | : : | : | : |
| | : | : |B| : | : |
| : : | : : : : | : |
| | : | : : : : | : |
| | : | : : : : | : |
|Y| : : : : | : : : |
+-------------------+
  (Barat)

Result
Episode : 3
Score : -185
Status : Belum Sukses


# Membuat Agen Cerdas menggunakan Q-Learning

**Q-Learning**  merupakan salah satu algoritma reinforcement learning yang berusaha untuk menemukan tindakan terbaik untuk dilakukan dalam suatu keadaan. Q-Learning berusaha mempelajari kebijakan yang dapat memaksimalkan reward/hadiah.

Disini q-learning melakukannya dengan menggunakan Q-table. Q table merupakan matrix yang memiliki bentuk ``states x actions``.

Saat training agen akan berinteraksi dengan lingkungannya berdasarkan dua cara, yaitu dengan "exploiting" dan "exploring". Exploiting berarti memanfaatkan informasi langkah yang paling baik diambil yang tersimpan di q-table untuk melakukan suatu langkah. Sedangkan dengan exploring agen akan mengambil langkah secara acak, hal ini dapat memungkinkan agen untuk menemukan langkah baru yang mungkin lebih baik dan menyimpannya di dalam q-table. Kita disini dapat menyeimbangkan exploitation/exploration dengan menggunakan parameter epsilon.

Di saat training, agen akan memperbarui q-table pada setiap langkah berdasarkan reward yang didapatkannya hingga episode tersebut berakhir. Proses ini akan dilakukan berulang-ulang hingga didapatkan nilai q (q-values) yang optimal. Nilai dari q-values dihitung menggunakan rumus matematika seperti berikut :

![rumus q](./rumusqlearning.png)

Disini ada beberapa parameter yang berperan dalam mengupdate q-values, yakni
* **Epsilon (ϵ)**, menyatakan seberapa banyak exploration yang ingin dilakukan
* **Learning rate atau alpha (α)**, yaitu seberapa banyak nilai baru akan diterima dibanding nilai lama.
* **Gamma (γ)**, untuk menyeimbangkan reward yang akan didapatkan dalam jangka pendek dan dalam jangka panjang.
* **Reward**, merupakan nilai yang diberikan ketika telah menyelesaikan suatu aksi.

<img src="https://storage.googleapis.com/lds-media/images/q-matrix-initialized-to-learned_gQq0BFs.width-1200.png" alt="intralogistic robot" width="500"/>

In [9]:
# Membuat Q Table
# dibuat dengan menggunakan numpy array dengan default value 0
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [10]:
# Mengatur hyperparameter untuk mengupdate q-table
alpha = 0.1
gamma = 0.6
epsilon = 0.1

In [23]:
# Melakukan training sebanyak total_episodes
total_episodes = 1000

for i in range(1, total_episodes+1):
    state = env.reset()
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:  
            action = env.action_space.sample()  # Melakukan eksplorasi/exploration
        else:
            action = np.argmax(q_table[state])  # Melakukan exploitasi/exploitation

        next_state, reward, done, info = env.step(action)  # Memberikan aksi ke lingkungan
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)  # Menghitung nilai baru untuk q-table
        q_table[state, action] = new_value  # Memberikan nilai baru ke q-table
        state = next_state

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Training.{'.' * (i%6)}")
        print(f"Episode: {i}/{total_episodes}")

print("\nTraining finished.\n")

Training.
Episode: 900/1000

Training finished.



In [79]:
# Save q_table untuk digunakan di kesempatan berikutnya
np.save("ModelAgen.npy", q_table)

In [12]:
del q_table

In [18]:
# Load q_table
q_table = np.load("ModelAgen.npy")

## Mengevaluasi performa agen

In [25]:
#Mengevaluasi performa dari agen

results = []
total_epochs, total_penalties, total_rewards = 0, 0, 0
episodes = 100

for eps in range(1, episodes+1):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1
        total_rewards += reward

    total_penalties += penalties
    total_epochs += epochs
    clear_output(wait=True)
    print(f"Q-Learning Agent, Current episodes : {eps}")

results.append("Q-Learning Agent")
results.append(f"Results after {episodes} episodes:")
results.append(f"Rata-rata timestep per episode: {total_epochs / episodes}")
results.append(f"Rata-rata penalti per episode: {total_penalties / episodes}")
results.append(f"Rata-rata reward per episode : {total_rewards / episodes}\n")

# Dibandingkan dengan random agent

total_epochs, total_penalties, total_rewards = 0, 0, 0

for eps in range(1, episodes+1):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1
        total_rewards += reward

    total_penalties += penalties
    total_epochs += epochs
    clear_output(wait=True)
    print(f"Random Agent, Current episodes : {eps}")


results.append("Random Agent")
results.append(f"Results after {episodes} episodes:")
results.append(f"Rata-rata timestep per episode: {total_epochs / episodes}")
results.append(f"Rata-rata penalti per episode: {total_penalties / episodes}")
results.append(f"Rata-rata reward per episode : {total_rewards / episodes}")

clear_output(wait=True)
for result in results:
    print(result)

Q-Learning Agent
Results after 100 episodes:
Rata-rata timestep per episode: 30.32
Rata-rata penalti per episode: 0.0
Rata-rata reward per episode : 0.68

Random Agent
Results after 100 episodes:
Rata-rata timestep per episode: 18647.63
Rata-rata penalti per episode: 6164.92
Rata-rata reward per episode : -74100.91


## Mencoba agen cerdas

In [33]:
# Mencoba agen secara live

def print_frames(frames):
    """Untuk memprint setiap frame dari gameplay, input berupa list berisi dictionary"""
    for i, frame in enumerate(frames):
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.4)
        if i != len(frames)-1:
            clear_output(wait=True)

epochs = 0
penalties, reward = 0, 0

frames = []  # for animation

done = False
state = env.reset()

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    }
    )

    epochs += 1

sleep(2)
clear_output(wait=True)
print_frames(frames)

print(f"\n\nFinal Result\\\n\tBanyak Langkah yang diambil: {epochs}")
print(f"\tPenalti Terjadi: {penalties}")

+-------------------+
|R| | : | : | : : :G|
| | | | : : | : : : |
| | : | : : | : | : |
| | : | : : | : | : |
| | : | : : | : | : |
| | : | : |B| : | : |
| : : | : : : : | : |
| | : | : : : : | : |
| | : | : : : : | : |
|Y| : : : : | : : : |
+-------------------+
  (Taruh Barang)

Timestep: 40
State: 185
Action: 5
Reward: 30


Final Result\
	Banyak Langkah yang diambil: 40
	Penalti Terjadi: 0


# Mencoba mengintip q-table

In [101]:
import pandas as pd

q_table_dataframe = pd.DataFrame(q_table)

In [115]:
q_table_dataframe

,0,1,2,3,4,5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-2.499993,-2.499988,-2.499988,-2.499988,-2.499980,-11.499988
2,-2.429255,-2.382091,-2.382091,-2.382091,-2.303485,-11.382091
3,-2.499287,-2.498812,-2.498812,-2.498812,-2.498020,-11.498812
4,-2.500000,-2.500000,-2.500000,-2.500000,-11.394862,-11.489276
...,...,...,...,...,...,...
1995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1996,-2.499179,-2.499219,-2.499219,-2.499177,-9.266367,-9.472631
1997,-2.291657,-2.291775,-2.299338,-2.299960,-4.248313,-3.664688
1998,-2.496316,-2.496223,-2.496477,-2.496024,-8.464097,-8.764241


In [137]:
# misal kita berada di state nomor 1996, 
#langkah yang akan diambil adalah dengan melihat kolom mana yang memiliki nilai terbesar, yakni :
action = np.argmax(q_table[1996])
print(action)

3


Dihasilkan nilai-nilai tersebut, beberapa posisi di tabel nilainya 0 semua, hal itu karena tempat tujuan dan tempat barang berasal tidak akan sama. Jadi keadaan ketika lokasi_barang dan tujuan sama tidak pernah terjadi. Disini tetap dibuat seolah2 ada untuk memudahkan encode dan decode keadaan lingkungan

In [123]:
# coba kita decode state tersebut
robot_row, robot_col, lokasi_barang, tujuan = env.decode(0)
print(f"robot_row : {robot_row}\nrobot_col : {robot_col}\nlokasi barang : {lokasi_barang}\ntujuan : {tujuan}")
env.s = 0
env.render()
print(f"Nilai q-table : {q_table[0]}")

robot_row : 0
robot_col : 0
lokasi barang : 0
tujuan : 0
+-------------------+
|R| | : | : | : : :G|
| | | | : : | : : : |
| | : | : : | : | : |
| | : | : : | : | : |
| | : | : : | : | : |
| | : | : |B| : | : |
| : : | : : : : | : |
| | : | : : : : | : |
| | : | : : : : | : |
|Y| : : : : | : : : |
+-------------------+
  (Taruh Barang)
Nilai q-table : [0. 0. 0. 0. 0. 0.]


# JOB DESK

#### Dimas Tri Mustakim (205150200111049)

1. Mencari dan mendiskusikan ide
2. Membuat program
3. Membuat makalah

#### Salsabila Dita Prasetya (205150201111024)

1. Mendiskusikan ide
2. Membuat makalah